### Ejercicio 15: Vacunas

Una compañía distribuidora de vacunas dispone de un total de un millón de vacunas para repartir en España. Las vacunas llegan a través de Barajas, por lo que tienen un coste asociado al transporte hasta el lugar de su utilización (en euros por dosis). Cada comunidad autónoma tiene un número de individuos susceptibles de ser vacunados y realiza una oferta por dosis de vacuna (en euros por dosis). El requisito del Gobierno para que se comercialicen las vacunas es que al menos se vacune al 15% de la población susceptible en cada territorio. No se desea vacunar a más del 40% de la población de un territorio, puesto que ya se considera conseguida la inmunidad de grupo. ¿Cuál es el beneficio máximo que puede obtener la empresa? Los datos por comunidades autónomas están recogidos en el Excel adjunto “DatosVacunas.xlsx”

#### Importamos Pyomo

In [ ]:
from pyomo.environ import *
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_excel('Datos/DatosVacunas.xlsx',index_col=0)
df

In [ ]:
susceptibles = df['Individuos susceptibles']
pagan = df['Pagan (eur/dosis)']
transp = df['Transporte (eur/dosis)']
comunidades = df.index.values.tolist()

#### Creamos un problema concreto, es decir, un objeto de la clase problema que ya contenga parámetros

In [ ]:
model = ConcreteModel()

#### Necesitamos utilizar sets

In [ ]:
model.com=Set(initialize=comunidades)


#### Definimos las variables

In [ ]:
model.vacunas=Var(model.com,within=NonNegativeReals)


#### Definimos la función objetivo

In [ ]:
model.obj = Objective(expr = sum(model.vacunas[c]*(pagan[c]-transp[c]) for c in model.com ), sense=maximize)

#### Definimos la restricciones

In [ ]:
model.minvac = ConstraintList()
for c in model.com:
    model.minvac.add(
        model.vacunas[c]>=0.15*susceptibles[c]
    )

model.maxvac = ConstraintList()
for c in model.com:
    model.maxvac.add(
        model.vacunas[c]<=0.4*susceptibles[c]
    )

model.totalvacunas = Constraint(expr = sum(model.vacunas[c] for c in model.com)<=1000000)


#### Resolvemos el problema

In [ ]:
results = SolverFactory('glpk').solve(model)
model.pprint()
results.write()   

#### Exportamos a Excel

In [ ]:
df2=pd.DataFrame.from_dict(model.vacunas.extract_values(),orient='index')
df2.to_excel('Resultados/Result_Vacunas.xlsx')
df2

In [ ]:
vac=df2.to_numpy()
vac
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8, 8), dpi=300)
plt.plot(comunidades,vac,'o',color='black')
plt.xticks(rotation=90)
plt.xlabel('Comunidad')
plt.ylabel('Vacunas recibidas')
plt.savefig('Resultados/figuravacunas.svg')


